In [2]:
!pip install pyrho

     |████████████████████████████████| 1.6 MB 474 kB/s eta 0:00:01
     |████████████████████████████████| 306 kB 5.1 MB/s eta 0:00:01
  Using cached jedi-0.18.0-py2.py3-none-any.whl (1.4 MB)
  Using cached parso-0.8.2-py2.py3-none-any.whl (94 kB)
     |████████████████████████████████| 98 kB 2.4 MB/s eta 0:00:011
  Created wheel for pyrho: filename=PyRhO-0.9.4-py3-none-any.whl size=1644973 sha256=f6c2014d8d4dfe08d72c4d6898529682141f9026e2b2f78ed30ba3e287563cf7
  Stored in directory: /home/maria/.cache/pip/wheels/0f/ae/4c/3820021bd79d4781f37c3c173eca51e454feec9525c2d85e30
  Created wheel for lmfit: filename=lmfit-1.0.2-py3-none-any.whl size=85274 sha256=0e66f09c12e84787b72a385079a366f4741feb550b7115f559116e4ffe740786
  Stored in directory: /home/maria/.cache/pip/wheels/42/81/51/72ff0f8e216caa9fe1e9dd1e7c287274b1eb75c79bfc858f5b
  Created wheel for asteval: filename=asteval-0.9.25-py3-none-any.whl size=17301 sha256=26f3f9147a876c23f48c33b0ce7a14353db806ca8c47e7966821510a880b6308
  Stor

In [3]:
from pyrho import *

In [6]:
### Define the rhodopsin model
nStates = '6'
#origParams = loadData('fitted{}sParams'.format(nStates))
origParams = modelFits[nStates]['ChR2'] # Load from pre-fit models
RhO = models[nStates](origParams)

### Define the stimulation protocol
Prot = protocols['step']()
Prot.phis = [1e18]
Prot.Vs = [None]
Prot.cycles = [[150.,100.],[200.,75.]]

In [7]:
spikes=np.load('/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/spk_arr2.npy')
cell_identity=np.load('/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/cell_identity2.npy')

In [34]:
from brian2 import *
start=time.time()
defaultclock.dt = 0.1*ms
tau = 10*ms
vr = -70*mV
vt = -60*mV
eqs = '''
dv/dt = I_ampa/Cm : volt
I_ampa=-g_ext_ampa*s_ext*(v-E_ampa) : amp
ds_ext/dt=-s_ext/t_ampa : 1
'''
eqs_rho = '''
dv/dt = I_ampa/Cm +((-I*R_m)+E_m-v)/tau_m : volt
I_ampa=-g_ext_ampa*s_ext*(v-E_ampa) : amp
ds_ext/dt=-s_ext/t_ampa : 1
'''+ RhO.brian_phi_t
netParams = {'t_ampa':2*ms, 'Cm':0.5*nF, 'E_ampa':0.0*mV, 'g_ext_ampa':3.1*nS,'vt':-60*mV,'vr':-70*mV,'w':1,'E_m':-70*mV,'tau_m':10*ms, 'R_m':70*Mohm}
E_ampa=0.0*mV
neuron = NeuronGroup(100, eqs, threshold='v>vt', reset='v=vr')
neuron_rho=NeuronGroup(100,eqs_rho,threshold='v>vt',reset='v=vr',name='Inputs')
neuron.v = -10*mV
neuron.s_ext = 0
spikes_gen=SpikeGeneratorGroup(1000,list(cell_identity),spikes*1000*ms)
w = 1
S = Synapses(spikes_gen, neuron, on_pre='s_ext += w')
S.connect(p=0.1)
S = Synapses(spikes_gen, neuron_rho, on_pre='s_ext += w')
S.connect(p=0.1)
S = Synapses(neuron_rho, neuron, on_pre='s_ext += w')
S.connect(p=0.5)
# Record the spikes
Mgp = SpikeMonitor(neuron)
Mgp_rho = SpikeMonitor(neuron_rho)
Minput = SpikeMonitor(spikes_gen)
monitors={'states' : StateMonitor(neuron_rho, RhO.brianStateVars, record=0), 'I': StateMonitor(neuron_rho, 'I', record=0),                  # Record current
            'spikes':[Mgp,Mgp_rho,Minput]}

In [31]:
RhO.brian_phi_t

'\n            dC_1/dt = Gr0*C_2 + Gd1*O_1 - Ga1*C_1 : 1\n            dI_1/dt = Ga1*C_1 - Go1*I_1 : 1\n            dO_1/dt = Go1*I_1 - (Gd1+Gf)*O_1 + Gb*O_2 : 1\n            dO_2/dt = Go2*I_2 + Gf*O_1 - (Gd2+Gb)*O_2 : 1\n            dI_2/dt = Ga2*C_2 - Go2*I_2 : 1\n            C_2 = 1 - C_1 - I_1 - O_1 - O_2 - I_2 : 1\n            H_p = Theta*((phi(t)**p)/(phi(t)**p+phi_m**p)) : 1\n            H_q = Theta*((phi(t)**q)/(phi(t)**q+phi_m**q)) : 1\n            Ga1 = k1*H_p : second**-1\n            Ga2 = k2*H_p : second**-1\n            Gf = Gf0 + k_f*H_q : second**-1\n            Gb = Gb0 + k_b*H_q : second**-1\n            f_v = (1-exp(-(v-E)/v0))/((v-E)/v1) : 1\n            f_phi = O_1+gam*O_2 : 1\n            I = g0*f_phi*f_v*(v-E) : amp\n            Theta = int(phi(t) > 0*phi(t)) : 1 (shared)\n            '

In [35]:
### Run the simulation
import time
start=time.time()
import brian2 as br
net = br.Network(br.collect())
net.add(monitors.values())
Sim = simulators['Brian'](Prot, RhO, simParams['Brian'], net, netParams, monitors)
Sim.run()
Sim.plot()
end=time.time()
print(end-start)


Running 'step' protocol with Brian for the Six-state ChR2 model... 

{nRuns=1, nPhis=1, nVs=1}


BrianObjectException: Error encountered with object named "statemonitor_3".
Object was created here (most recent call only, full details in debug log):
  File "<ipython-input-34-550b4b2f4727>", line 35, in <module>
    monitors={'states' : StateMonitor(neuron_rho, RhO.brianStateVars, record=0), 'I': StateMonitor(neuron_rho, 'I', record=0),                  # Record current

An exception occured during the execution of the run block of code object statemonitor_3_codeobject.
 (See above for original error message and traceback.)